In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

In [41]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/sample_submission.csv')

In [42]:
ss = StandardScaler()
train['X1'] = ss.fit_transform(train[['X1']])
test['X1'] = ss.transform(test[['X1']])

train['X2'] = ss.fit_transform(train[['X2']])
test['X2'] = ss.transform(test[['X2']])

train['X3'] = ss.fit_transform(train[['X3']])
test['X3'] = ss.transform(test[['X3']])

train['X10'] = ss.fit_transform(train[['X10']])
test['X10'] = ss.transform(test[['X10']])

In [43]:
train.corr()[abs((train.corr()*100))['label'] < 1].index

Index(['X26', 'X37', 'X51'], dtype='object')

In [44]:
(train.corr()*100)

,label,user_id,X1,X2,X3,X4,X5,X6,X7,X8,...,X42,X43,X44,X45,X46,X47,X48,X49,X50,X51
label,100.000000,6.187620,-3.269290,1.358753,2.922354,-16.098274,-12.090062,11.879348,-11.979204,4.860493,...,-4.657551,-2.376109,-2.263530,-1.097813,NaN,NaN,NaN,-9.828567,-7.636048,0.177072
user_id,6.187620,100.000000,1.395712,1.496866,-13.824336,29.580326,22.293786,72.346516,22.109833,-11.480707,...,-0.101536,0.772277,2.124226,3.927746,NaN,NaN,NaN,-1.352059,25.835625,-0.279711
X1,-3.269290,1.395712,100.000000,-0.715866,-0.551261,7.408420,7.429416,0.121983,1.630698,-1.100358,...,2.179736,0.144010,1.203981,-0.741735,NaN,NaN,NaN,4.365022,1.571228,0.000263
X2,1.358753,1.496866,-0.715866,100.000000,4.725665,1.153869,4.034182,1.833937,4.258145,-4.206818,...,-0.167350,-1.347701,-1.156921,-3.260232,NaN,NaN,NaN,2.125600,0.862687,-0.024310
X3,2.922354,-13.824336,-0.551261,4.725665,100.000000,-33.479459,-31.880842,-15.790539,-18.825467,8.058668,...,1.614811,-0.108062,-1.085067,-3.894903,NaN,NaN,NaN,3.263184,-19.194276,0.301339
X4,-16.098274,29.580326,7.408420,1.153869,-33.479459,100.000000,77.791460,21.933396,49.034587,-33.923937,...,4.945026,4.013070,7.787042,5.689933,NaN,NaN,NaN,25.847212,49.216760,-1.363746
X5,-12.090062,22.293786,7.429416,4.034182,-31.880842,77.791460,100.000000,14.799095,30.601435,-25.123751,...,3.826574,-1.020974,4.691023,4.995300,NaN,NaN,NaN,11.446727,33.917788,-0.620779
X6,11.879348,72.346516,0.121983,1.833937,-15.790539,21.933396,14.799095,100.000000,9.453063,-23.677304,...,-2.118734,-1.164294,1.267234,2.910085,NaN,NaN,NaN,-4.690491,13.960835,-0.098438
X7,-11.979204,22.109833,1.630698,4.258145,-18.825467,49.034587,30.601435,9.453063,100.000000,-3.205093,...,0.001258,2.299679,5.873306,3.970228,NaN,NaN,NaN,10.929970,38.817586,-0.892832
X8,4.860493,-11.480707,-1.100358,-4.206818,8.058668,-33.923937,-25.123751,-23.677304,-3.205093,100.000000,...,-0.853608,-1.707810,-1.754722,-1.937195,NaN,NaN,NaN,-7.591775,-23.983390,0.645535


In [45]:
train['X13'].fillna(train['X13'].median(), inplace = True)
# train = train.drop(['user_id', 'X26', 'X37', 'X51'], axis = 1)
# test = test.drop(['user_id', 'X26', 'X37', 'X51'], axis = 1)

y = train.pop('label')
X = train

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [47]:
model = XGBClassifier()
params = {'learning_rate': [0.2, 0.4, 0.6, 0.8, 1.0],
         'max_depth': [2,4,6,8,10],
          'min_child_weight': [1,3,5,7],
          'gamma': [0, 0.1, 0.2, 0.3, 0.4, 0.5],
          'colsample_bytree': [0.2, 0.4, 0.6]
         }
model1 = RandomizedSearchCV(model, params, n_iter = 5, cv = 5, random_state = 10)
model2 = RandomizedSearchCV(model, params, n_iter = 5, cv = 5, random_state = 10)

In [48]:
model1.fit(X_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, gamma=None,
                                           gpu_id=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=None, max_bin=None,...
                                           monotone_constraints=None,
                                           n_estimators=100, n_jobs=

In [49]:
pred = model1.predict(X_test)
score = f1_score(pred, y_test)
print('score: ', score)

score:  0.7007692307692308


In [ ]:
model2.fit(X, y)

In [ ]:
pred = model.predict(X_test)
score = f1_score(pred, y_test)
print('score: ', score)

In [20]:
prediction = model.predict(test)
submission['prediction']  = prediction
submission.pop('label')
submission.to_csv('spammer_submission1.csv', index = False)

In [21]:
submission

,user_id,prediction
0,57,0
1,58,0
2,59,0
3,60,0
4,61,0
...,...,...
24143,482942,0
24144,482943,0
24145,482944,0
24146,482945,0
